In [1]:
from datetime import datetime

def datetimeFromUnixTime(t):
    ts = int(t)
    return datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')

def getCurrentTime():
    return str(datetime.now())

print(datetimeFromUnixTime(1557397225))
print(getCurrentTime())

2019-05-09 10:20:25
2019-05-12 00:38:30.650481


In [26]:
import mysql.connector

engine = mysql.connector.connect(
    user="ecohydro",
    password="7fQh53G6D2BuCnT54hXH95CJx",
    host="ubc-ecohydro-nodes-mysql-database.cycweraudmq1.us-east-1.rds.amazonaws.com",
    port='3306',
    database='eco_nodes'
)

#insertDataQuery = 'datacall eco_nodes.insert_data(%s, {boot_count}, {sensor}, {param}, {read_time}, {store_time}, {reading});


data = "02&1557397229&0&2:15266.1,21.7&3:100000.0,21.2&17:1.41,21.5,1&22:1.76,21.4,1"


def writeDataToDatabase(node, boot_count, sensor, param, read_time, store_time, reading):
    #cur.execute(f'CREATE TABLE {node} (id serial PRIMARY KEY, num float, data varchar);')
    print(f'call eco_nodes.insert_data({node}, {boot_count}, {sensor}, {param}, \'{read_time}\', \'{store_time}\', {reading});')
    cur.execute(f'call eco_nodes.insert_data({node}, {boot_count}, {sensor}, {param}, \'{read_time}\', \'{store_time}\', {reading});')
    #cur.execute("SELECT * FROM test;")
    #cur.execute(insertDataQuery, values)
    engine.commit()


def parseDataString(string):
    entries = string.split('&')
        
    node = entries[0]
    read_time = datetimeFromUnixTime(entries[1])
    store_time = getCurrentTime()
    
    boot_count = entries[2]
    
    for entry in entries[3:]:
        e = entry.split(':')
        
        sensor = e[0]
        readings = e[1].split(',')
        
        for param, reading in enumerate(readings):
            #writeDataToDatabase(int(node), int(boot_count), int(sensor), param, read_time, store_time, float(reading));
            writeDataToDatabase(node, boot_count, sensor, param, read_time, store_time, reading);

parseDataString(data)

cur.close()
engine.close()

call eco_nodes.insert_data(02, 0, 2, 0, '2019-05-09 10:20:29', '2019-05-12 00:56:41.603545', 15266.1);
call eco_nodes.insert_data(02, 0, 2, 1, '2019-05-09 10:20:29', '2019-05-12 00:56:41.603545', 21.7);
call eco_nodes.insert_data(02, 0, 3, 0, '2019-05-09 10:20:29', '2019-05-12 00:56:41.603545', 100000.0);
call eco_nodes.insert_data(02, 0, 3, 1, '2019-05-09 10:20:29', '2019-05-12 00:56:41.603545', 21.2);
call eco_nodes.insert_data(02, 0, 17, 0, '2019-05-09 10:20:29', '2019-05-12 00:56:41.603545', 1.41);
call eco_nodes.insert_data(02, 0, 17, 1, '2019-05-09 10:20:29', '2019-05-12 00:56:41.603545', 21.5);
call eco_nodes.insert_data(02, 0, 17, 2, '2019-05-09 10:20:29', '2019-05-12 00:56:41.603545', 1);
call eco_nodes.insert_data(02, 0, 22, 0, '2019-05-09 10:20:29', '2019-05-12 00:56:41.603545', 1.76);
call eco_nodes.insert_data(02, 0, 22, 1, '2019-05-09 10:20:29', '2019-05-12 00:56:41.603545', 21.4);
call eco_nodes.insert_data(02, 0, 22, 2, '2019-05-09 10:20:29', '2019-05-12 00:56:41.603545

In [1]:

from http.server import BaseHTTPRequestHandler
#from socketserver import ThreadingMixIn
#import threading
import cgi

import mysql.connector


class PostHandler(BaseHTTPRequestHandler):
    
    # Initialize variables for communicating with data-base
    def __init__(self):
       
        self.engine = mysql.connector.connect(
            user="ecohydro",
            password="7fQh53G6D2BuCnT54hXH95CJx",
            host="ubc-ecohydro-nodes-mysql-database.cycweraudmq1.us-east-1.rds.amazonaws.com",
            port='3306',
            database='eco_nodes'
        )
        
        self.cur = engine.cursor()

    
    def do_POST(self):
        # Parse the form data posted
        form = cgi.FieldStorage(
            fp=self.rfile, 
            headers=self.headers,
            environ={'REQUEST_METHOD':'POST',
                     'CONTENT_TYPE':self.headers['Content-Type'],
                     })

        # Echo back information about what was posted in the form
        for field in form.keys():
            field_item = form[field]
            # Regular form value
            print('\t%s=%s\n' % (field, form[field].value))
        return
        
        
      #  if string[0] == 't':
      #      parseTitleString(string)
      #  else:
      #      parseDataString(string)
        
    
    def writeDataToDatabase(node, boot_count, sensor, param, read_time, store_time, reading):
        print(f'call eco_nodes.insert_data({node}, {boot_count}, {sensor}, {param}, \'{read_time}\', \'{store_time}\', {reading});')
        self.cur.execute(f'call eco_nodes.insert_data({node}, {boot_count}, {sensor}, {param}, \'{read_time}\', \'{store_time}\', {reading});')
        self.engine.commit()
    
    def parseDataString(string):
        entries = string.split('&')

        node = entries[0]
        read_time = datetimeFromUnixTime(entries[1])
        store_time = getCurrentTime()

        boot_count = entries[2]

        for entry in entries[3:]:
            e = entry.split(':')

            sensor = e[0]
            readings = e[1].split(',')

            for param, reading in enumerate(readings):
                #writeDataToDatabase(int(node), int(boot_count), int(sensor), param, read_time, store_time, float(reading));
                writeDataToDatabase(node, boot_count, sensor, param, read_time, store_time, reading);
    
    def parseTitleString(string):
        print("title string: " + string)
    
    # Cleanup variables for talking to database
    def __del__(self):
        this.cur.close()
        this.engine.close()
    
    
if __name__ == '__main__':
    from http.server import HTTPServer
    server = HTTPServer(('0.0.0.0', 5200), PostHandler)
    print('Starting server, use <Ctrl-C> to stop')
    server.serve_forever()

NameError: name 'BaseHTTPRequestHandler' is not defined

In [ ]:


from flask import Flask, request

app = Flask(__name__)

@app.route("/", methods=['POST', 'GET'])
def handleData():
        if request.method == 'POST':
                jsonData = request.get_json()
                print(jsonData['payload_fields']['data'])
                return '...'
        return "Running and ready"


if __name__ == '__main__':
        app.run(debug=True, host='0.0.0.0', port=5200)

